## It's the last day of the Gaia sprint!

In [8]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
# %matplotlib notebook

In [2]:
import apogee.tools.read as apread
from apogee.tools import apStarWavegrid

In [3]:
apStarWavegrid()

array([ 15100.80154164,  15101.01016837,  15101.21879797, ...,
        16999.33764336,  16999.57249951,  16999.80735891])

In [4]:
apmatch = pd.read_csv('apogee_gaia_rvcut.csv')

In [14]:
with PdfPages('specplot_apmatch.pdf') as pdf:
    for fidx in np.arange(0, len(apmatch['APOGEE_ID']), 25):
        fig = plt.figure(figsize=(10,7))
        ax = fig.add_subplot(111)
        # Turn off axis lines and ticks of the big subplot
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        for idx, (loc_id, apogee_id) in enumerate(zip(apmatch['LOCATION_ID'][fidx:fidx+25], apmatch['APOGEE_ID'][fidx:fidx+25])):
            try:
                spec, hdr = apread.apStar(loc_id, apogee_id, ext=1)
            except OSError:  # usually happens for 1m targets because locationID isn't used for apStar lookup
                continue
            smallax = fig.add_subplot(5, 5, idx+1)
            plt.subplots_adjust(wspace=0, hspace=0)
            if idx < 20:
                smallax.axes.xaxis.set_ticklabels([])
            smallax.axes.yaxis.set_ticklabels([])
            #plt.xlim([15720, 15790])
            if len(spec) < 8575:  # more than one visit, plot combined "global weighting" spectrum in orange
                #plt.plot(apStarWavegrid()[2900:3200], spec[1][2900:3200], color='C1')
                offset = 0
                for visit in spec[2:]:
                    plt.plot(apStarWavegrid()[2900:3200], visit[2900:3200] + offset, lw=1)
                    offset += (plt.ylim()[1] - plt.ylim()[0])*0.10
            else:  # only one visit, plot it in blue
                plt.plot(apStarWavegrid()[2900:3200], spec[2900:3200], color='C0')
            plt.text(15720, plt.ylim()[0] + 0.05*(plt.ylim()[1] - plt.ylim()[0]), apogee_id)
        ax.tick_params(top='off', bottom='off', left='off', right='off')
        ax.axes.xaxis.set_ticklabels([])
        ax.axes.yaxis.set_ticklabels([])
        ax.set_xlabel('Wavelength (\AA)', labelpad=15)
        ax.set_ylabel('Uncalibrated apStar flux', labelpad=10)
        pdf.savefig(fig)